# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
cities = pd.read_csv("cities.csv")
cities.head()

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mathbaria,22.29,89.96,81.77,90,100,4.12,BD,1597097384
1,1,ushuaia,-54.80,-68.30,33.80,80,40,18.34,AR,1597097384
2,2,fengxian,34.70,116.59,75.90,94,23,3.67,CN,1597097384
3,3,saint george,37.10,-113.58,102.99,7,1,8.05,US,1597097253
4,4,srednekolymsk,67.45,153.68,46.44,63,59,7.38,RU,1597097385


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
gmaps.configure(api_key=g_key)

In [33]:
locations = cities[["Lat", "Lon"]]
humidity = cities['Humidity'].astype(float)
maxhumidity = humidity.max()

In [34]:
figure_layout = {
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}


In [35]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2, layout = figure_layout)
Heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=2)
fig.add_layer(Heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px'))

In [36]:
cities = cities.drop(columns='Unnamed: 0')
cities.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mathbaria,22.29,89.96,81.77,90,100,4.12,BD,1597097384
1,ushuaia,-54.80,-68.30,33.80,80,40,18.34,AR,1597097384
2,fengxian,34.70,116.59,75.90,94,23,3.67,CN,1597097384
3,saint george,37.10,-113.58,102.99,7,1,8.05,US,1597097253
4,srednekolymsk,67.45,153.68,46.44,63,59,7.38,RU,1597097385


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
hotel_df = cities[(cities['Max Temp']>=70) & (cities['Max Temp']<=80) \
                      & (cities['Wind Speed'] < 10) & (cities['Cloudiness']==0)]
hotel_df.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,olhao,37.03,-7.84,78.80,47,0,5.82,PT,1597097396
115,karacakoy,41.40,28.38,77.00,73,0,9.17,TR,1597097413
131,ajdabiya,30.76,20.23,76.82,75,0,9.71,LY,1597097418
161,tukrah,32.53,20.58,77.85,67,0,8.34,LY,1597097426
219,trapani,37.83,12.67,78.80,88,0,5.82,IT,1597097442


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df['Hotel Name'] = ""

<ipython-input-38-55c2a054e183>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


In [39]:
hotel_df.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,olhao,37.03,-7.84,78.80,47,0,5.82,PT,1597097396,
115,karacakoy,41.40,28.38,77.00,73,0,9.17,TR,1597097413,
131,ajdabiya,30.76,20.23,76.82,75,0,9.71,LY,1597097418,
161,tukrah,32.53,20.58,77.85,67,0,8.34,LY,1597097426,
219,trapani,37.83,12.67,78.80,88,0,5.82,IT,1597097442,


In [40]:
for index,row in hotel_df.iterrows():
    lat = row.Lat
    lng = row.Lon
    
    url= f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=lodging&key={g_key}'
    response = requests.get(url).json()
    try:
        hotel_df.loc[index,'Hotel Name']=response['results'][0]['name']
    except(KeyError,IndexError):
        print('Missing field/result... skipping.')
    


C:\Users\Stephanie\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [41]:
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,olhao,37.03,-7.84,78.80,47,0,5.82,PT,1597097396,Zenit Guest House
131,ajdabiya,30.76,20.23,76.82,75,0,9.71,LY,1597097418,الحاج سليمان الشترة
161,tukrah,32.53,20.58,77.85,67,0,8.34,LY,1597097426,استراحة لاقروتا
231,nurota,40.56,65.69,78.80,27,0,4.70,UZ,1597097446,Guest House Ruslan Nurata & Tours
253,arys,42.43,68.80,77.00,36,0,4.47,KZ,1597097451,"Gostinitsa ""Madani"""
353,sao francisco,-15.95,-44.86,73.18,38,0,4.59,BR,1597097482,Pousada Peixe Vivo
460,sabzevar,36.21,57.68,77.47,26,0,8.41,IR,1597097512,پارکینگ وسایل سنگین جنوب
466,brindisi,40.58,17.67,79.00,83,0,3.36,IT,1597097513,Agriturismo Tibitone
548,qunduz,36.73,68.86,77.14,28,0,2.77,AF,1597097534,قريشي مارکيٹ


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [43]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px'))